In [1]:
import os
import asyncio
import json
import re
import textwrap
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.connectors.search_engine import GoogleConnector
from semantic_kernel.core_plugins import WebSearchEnginePlugin
from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
from pydantic import BaseModel, Field, ValidationError
from typing import List
from IPython.display import display, Markdown



load_dotenv()

True

In [ ]:

# import logging
# logging.basicConfig(level=logging.INFO)

In [3]:
observability = False
if observability :
    import asyncio
import logging

from opentelemetry._logs import set_logger_provider
from opentelemetry.exporter.otlp.proto.grpc._log_exporter import OTLPLogExporter
from opentelemetry.exporter.otlp.proto.grpc.metric_exporter import OTLPMetricExporter
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.metrics import set_meter_provider
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.sdk.metrics.view import DropAggregation, View
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.semconv.resource import ResourceAttributes
from opentelemetry.trace import set_tracer_provider




# Endpoint to the Aspire Dashboard
endpoint = "http://localhost:4317"

# Create a resource to represent the service/sample
resource = Resource.create({ResourceAttributes.SERVICE_NAME: "telemetry-aspire-dashboard-quickstart"})


def set_up_logging():
    exporter = OTLPLogExporter(endpoint=endpoint)

    # Create and set a global logger provider for the application.
    logger_provider = LoggerProvider(resource=resource)
    # Log processors are initialized with an exporter which is responsible
    # for sending the telemetry data to a particular backend.
    logger_provider.add_log_record_processor(BatchLogRecordProcessor(exporter))
    # Sets the global default logger provider
    set_logger_provider(logger_provider)

    # Create a logging handler to write logging records, in OTLP format, to the exporter.
    handler = LoggingHandler()
    # Add filters to the handler to only process records from semantic_kernel.
    handler.addFilter(logging.Filter("semantic_kernel"))
    # Attach the handler to the root logger. `getLogger()` with no arguments returns the root logger.
    # Events from all child loggers will be processed by this handler.
    logger = logging.getLogger()
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)


def set_up_tracing():
    exporter = OTLPSpanExporter(endpoint=endpoint)

    # Initialize a trace provider for the application. This is a factory for creating tracers.
    tracer_provider = TracerProvider(resource=resource)
    # Span processors are initialized with an exporter which is responsible
    # for sending the telemetry data to a particular backend.
    tracer_provider.add_span_processor(BatchSpanProcessor(exporter))
    # Sets the global default tracer provider
    set_tracer_provider(tracer_provider)


def set_up_metrics():
    exporter = OTLPMetricExporter(endpoint=endpoint)

    # Initialize a metric provider for the application. This is a factory for creating meters.
    meter_provider = MeterProvider(
        metric_readers=[PeriodicExportingMetricReader(exporter, export_interval_millis=5000)],
        resource=resource,
        views=[
            # Dropping all instrument names except for those starting with "semantic_kernel"
            View(instrument_name="*", aggregation=DropAggregation()),
            View(instrument_name="semantic_kernel*"),
        ],
    )
    # Sets the global default meter provider
    set_meter_provider(meter_provider)


# This must be done before any other telemetry calls
set_up_logging()
set_up_tracing()
set_up_metrics()



  

In [4]:
class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class Article(BaseModel):
    title: str
    content: str
    call_to_action: str

class ContentOutput(BaseModel):
    article: Article
    social_media_posts: List[SocialMediaPost]

In [5]:
service_id: str = "default_service_id"
def create_kernel(service_id= service_id) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id))


    return kernel

kernel = create_kernel()

In [6]:
from agent_instructions import (
    MARKET_NEWS_MONITOR_AGENT_INSTRUCTIONS,
    DATA_ANALYST_AGENT_INSTRUCTIONS,
    CONTENT_CREATOR_AGENT_INSTRUCTIONS,
    QUALITY_ASSURANCE_AGENT_INSTRUCTIONS,
)

def format_instruction(instruction_template: str, subject: str) -> str:
    return instruction_template.format(subject=subject)

subject = "Inflation in the US and the impact on the stock market in 2024"

market_news_instr = format_instruction(MARKET_NEWS_MONITOR_AGENT_INSTRUCTIONS, subject)
data_analyst_instr = format_instruction(DATA_ANALYST_AGENT_INSTRUCTIONS, subject)
content_creator_instr = format_instruction(CONTENT_CREATOR_AGENT_INSTRUCTIONS, subject)
quality_assurance_instr = format_instruction(QUALITY_ASSURANCE_AGENT_INSTRUCTIONS, subject)


In [7]:
# google_scraper_plugin.py
from semantic_kernel.functions import kernel_function
from googleapiclient.discovery import build
import requests
from bs4 import BeautifulSoup
from typing import List
import os

class WebScraperPlugin:
    def __init__(self, api_key: str, cse_id: str):
        self.api_key = api_key
        self.cse_id = cse_id

    @kernel_function(
        name="google_search",
        description="Performs a Google search and returns a list of URLs related to the query."
    )
    async def google_search(self, query: str, num_results: int = 5) -> List[str]:
        service = build("customsearch", "v1", developerKey=self.api_key)
        res = service.cse().list(q=query, cx=self.cse_id, num=num_results).execute()
        urls = [item['link'] for item in res.get("items", [])]
        return urls

    @kernel_function(
        name="scrape_url",
        description="Scrapes a given URL and returns the title and the first few paragraphs of text."
    )
    async def scrape_url(self, url: str) -> dict:
        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
            }
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            title = soup.title.string.strip() if soup.title else "No Title"
            paragraphs = soup.find_all("p")
            text_content = "\n".join(p.get_text(strip=True) for p in paragraphs[:3])  # First 3 paragraphs

            return {
                "url": url,
                "title": title,
                "text": text_content
            }
        except Exception as e:
            return {
                "url": url,
                "error": str(e)
            }

web_scraper_tool = WebScraperPlugin(
    api_key=os.getenv("GOOGLE_API_KEY"),
    cse_id=os.getenv("GOOGLE_SEARCH_ENGINE_ID")
)


In [8]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
    Based on the conversation history, determine the next agent to take a turn. 
    Only return the name of the agent from the following list:
    - MarketNewsMonitor
    - DataAnalyst
    - ContentCreator
    - ContentOfficer

    Rules:
    - MarketNewsMonitor starts first.
    - After MarketNewsMonitor, DataAnalyst speaks.
    - After DataAnalyst, ContentCreator speaks.
    - After ContentCreator, ContentOfficer speaks.
    - If ContentOfficer requests changes, restart the cycle with MarketNewsMonitor.

    History:
    {{$history}}
    """,
)

In [9]:
termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the structured content output (article and social_media_posts) is complete and valid JSON. 
    If the output is complete and valid, respond with "yes". Otherwise, respond with "no".

    History:
    {{$history}}
    """
)

In [10]:
output_settings = AzureChatPromptExecutionSettings(response_format=ContentOutput)


In [11]:
market_news_agent = ChatCompletionAgent(kernel=kernel,
                                         name="MarketNewsMonitor",
                                           instructions=market_news_instr,
                                            plugins=[web_scraper_tool]
                                           )
data_analyst_agent = ChatCompletionAgent(kernel=kernel, 
                                         name="DataAnalyst",
                                          instructions=data_analyst_instr,
                                          plugins=[web_scraper_tool])
content_creator_agent = ChatCompletionAgent(kernel=kernel, name="ContentCreator", instructions=content_creator_instr)
quality_assurance_agent = ChatCompletionAgent(kernel=kernel,
                                                 name="ContentOfficer",
                                                 instructions=quality_assurance_instr,
                                                 arguments=KernelArguments(settings=output_settings))

agent_group = AgentGroupChat(
    agents=[market_news_agent, data_analyst_agent, content_creator_agent, quality_assurance_agent],
    termination_strategy = KernelFunctionTerminationStrategy(
        agents=[quality_assurance_agent],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: any("yes" in item.content.lower() for item in result.value) if isinstance(result.value, list) else "yes" in result.value.content.lower(),
        history_variable_name="history",
        maximum_iterations=10
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: result.value[0].content.strip() if isinstance(result.value, list) and result.value else result.value.content.strip(),
        agent_variable_name="agents",
        history_variable_name="history"
    )
)

In [12]:
async def main(subject: str):

    initial_message = f"Subject: {subject}\n\nCollaborate to produce structured content output matching the provided schema."
    await agent_group.add_chat_message(message=initial_message)

    final_response = ""
    async for content in agent_group.invoke():
        print(f"# {content.name}: {content.content}\n")
        final_response = content.content

    return final_response

In [13]:
final_output = await main("Inflation in the US and the impact on the stock market in 2024")

INFO:semantic_kernel.agents.group_chat.agent_chat:Adding `1` agent chat messages
INFO:semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy:Kernel Function Selection Strategy next method called, invoking function: , selection
INFO:semantic_kernel.functions.kernel_function:Function selection invoking.
INFO:httpx:HTTP Request: POST https://gpt4-assistants-api.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21 "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=4, prompt_tokens=164, total_tokens=168, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.functions.kernel_function:Function selection succeeded.
INFO:semantic_kernel.functions.kernel_function:Function co

# MarketNewsMonitor: ### Summary Report: Inflation in the US and the Impact on the Stock Market in 2024

#### Key Insights:
1. **US Consumer Confidence**:
   - The Conference Board has been providing insights on economic indicators, emphasizing US consumer confidence. This metric often reflects inflationary pressures and can affect consumer spending, a critical driver of the economy.

2. **Inflation’s Impact on Investments**:
   - An article from U.S. Bank discusses the wide-ranging effects of inflation on investment portfolios, touching on shifts in asset allocation strategies that may be necessitated by inflation in 2024. Rising inflation can erode purchasing power and impact bond markets, potentially leading investors to increase allocations in equities or commodities as hedges.

3. **US Economy Statistics and Trends**:
   - Data from USAFacts highlights trends and statistics, offering a comprehensive view of the economic landscape. Such metrics are essential for understanding the b

INFO:httpx:HTTP Request: POST https://gpt4-assistants-api.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21 "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=4, prompt_tokens=753, total_tokens=757, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.functions.kernel_function:Function selection succeeded.
INFO:semantic_kernel.functions.kernel_function:Function completed. Duration: 0.548330s
INFO:semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy:Kernel Function Selection Strategy next method completed: , selection, result: [ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BIco72No9Su4JysXE1kzGpYsNxjOz', choices=[Choice(finish_reason='stop', ind

# DataAnalyst: ### Summary Report: Inflation in the US and the Impact on the Stock Market in 2024

#### Key Insights:
1. **US Consumer Confidence**:
   - The Conference Board provides insights on economic indicators with a focus on US consumer confidence, which is significantly affected by inflationary pressures impacting consumer spending—a critical driver of the economy. [Source](https://www.conference-board.org/topics/consumer-confidence)

2. **Inflation’s Impact on Investments**:
   - Insights from U.S. Bank discuss inflation’s extended impact on investment portfolios, indicating necessary shifts in asset allocation strategies for 2024. With rising inflation, consumers might experience eroded purchasing power and changes in bond markets, pushing investors to consider equities or commodities as hedges. [Source](https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html)

3. **US Economy Statistics and Trends**:
   - USAFacts

INFO:httpx:HTTP Request: POST https://gpt4-assistants-api.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21 "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=3, prompt_tokens=1433, total_tokens=1436, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.functions.kernel_function:Function selection succeeded.
INFO:semantic_kernel.functions.kernel_function:Function completed. Duration: 0.526944s
INFO:semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy:Kernel Function Selection Strategy next method completed: , selection, result: [ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BIcoNRHldO7oPTehrOTS0QGsoI6oT', choices=[Choice(finish_reason='stop', i

# ContentCreator: Here are the content pieces for the topic of "Inflation in the US and the impact on the Stock Market in 2024," tailored for different platforms:

### Blog Post: Understanding Inflation in 2024 and Its Impact on the Stock Market

---

#### Inflationary Trends and Economic Impact
Inflation is a crucial economic indicator that reflects the overall increase in price levels within an economy. As we navigate through 2024, inflation in the United States is anticipated to remain a significant factor influencing both consumer confidence and spending. According to insights from the [Conference Board](https://www.conference-board.org/topics/consumer-confidence), such inflationary pressures can dampen consumer spending—a major component driving economic growth—and impact overall economic stability.

#### Investment Strategies Amid Rising Inflation
As inflation affects purchasing power and bond markets, investors need to develop strategies that efficiently hedge against these erod

INFO:httpx:HTTP Request: POST https://gpt4-assistants-api.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-10-21 "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=3, prompt_tokens=2182, total_tokens=2185, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
INFO:semantic_kernel.functions.kernel_function:Function selection succeeded.
INFO:semantic_kernel.functions.kernel_function:Function completed. Duration: 0.651619s
INFO:semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy:Kernel Function Selection Strategy next method completed: , selection, result: [ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-BIcolLrQxuVzLPpz5QM86IxWhBBEM', choices=[Choice(finish_reason='stop', i

# ContentOfficer: {"article":{"title":"Summary Report: Inflation in the US and the Impact on the Stock Market in 2024","content":"\n### Key Insights:\n\n1. **US Consumer Confidence**:\n   - The Conference Board provides insights on economic indicators with a focus on US consumer confidence, which is significantly affected by inflationary pressures impacting consumer spending—a critical driver of the economy. [Source](https://www.conference-board.org/topics/consumer-confidence)\n\n2. **Inflation’s Impact on Investments**:\n   - Insights from U.S. Bank discuss inflation’s extended impact on investment portfolios, indicating necessary shifts in asset allocation strategies for 2024. With rising inflation, consumers might experience eroded purchasing power and changes in bond markets, pushing investors to consider equities or commodities as hedges. [Source](https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html)\n\n3. **US Econo

In [14]:
final_output

'{"article":{"title":"Summary Report: Inflation in the US and the Impact on the Stock Market in 2024","content":"\\n### Key Insights:\\n\\n1. **US Consumer Confidence**:\\n   - The Conference Board provides insights on economic indicators with a focus on US consumer confidence, which is significantly affected by inflationary pressures impacting consumer spending—a critical driver of the economy. [Source](https://www.conference-board.org/topics/consumer-confidence)\\n\\n2. **Inflation’s Impact on Investments**:\\n   - Insights from U.S. Bank discuss inflation’s extended impact on investment portfolios, indicating necessary shifts in asset allocation strategies for 2024. With rising inflation, consumers might experience eroded purchasing power and changes in bond markets, pushing investors to consider equities or commodities as hedges. [Source](https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html)\\n\\n3. **US Economy Stati

In [15]:
def extract_json(text: str) -> dict:
    pattern = re.compile(r"```json\s*(\{.*\})\s*```", re.DOTALL)
    match = pattern.search(text)
    if match:
        return json.loads(match.group(1))
    return json.loads(text.strip())

output_data = extract_json(final_output)
article = output_data.get("article", {})
posts = output_data.get("social_media_posts", [])

article_md = f"# {article.get('title')}\n\n{article.get('content')}\n\n**{article.get('call_to_action')}**"
display(Markdown(article_md))

for post in posts:
    print(f"Platform: {post['platform']}")
    print(textwrap.fill(post['content'], width=60))
    print('-' * 60)

# Summary Report: Inflation in the US and the Impact on the Stock Market in 2024


### Key Insights:

1. **US Consumer Confidence**:
   - The Conference Board provides insights on economic indicators with a focus on US consumer confidence, which is significantly affected by inflationary pressures impacting consumer spending—a critical driver of the economy. [Source](https://www.conference-board.org/topics/consumer-confidence)

2. **Inflation’s Impact on Investments**:
   - Insights from U.S. Bank discuss inflation’s extended impact on investment portfolios, indicating necessary shifts in asset allocation strategies for 2024. With rising inflation, consumers might experience eroded purchasing power and changes in bond markets, pushing investors to consider equities or commodities as hedges. [Source](https://www.usbank.com/investing/financial-perspectives/investing-insights/how-does-inflation-affect-investments.html)

3. **US Economy Statistics and Trends**:
   - USAFacts provides statistical insights and trends offering an understanding of the economic landscape, highlighting broader demographic and market trends crucial for grasping the impact of inflation rates and economic growth projections. [Source](https://usafacts.org/economy/)

4. **Political Influences**:
   - Another perspective from U.S. Bank covers the effects of political shifts, such as those from presidential elections, on stock markets. Political transformations in 2024 could lead to market volatility, affecting perceptions and potentially causing shifts in fiscal policies influencing inflation. [Source](https://www.usbank.com/investing/financial-perspectives/market-news/how-presidential-elections-affect-the-stock-market.html)

5. **Market Outlook from J.P. Morgan**:
   - J.P. Morgan presents a market outlook with global insights and projections, essential for understanding how international economic climates might impact US inflation and stock markets in 2024. International trends, particularly in major economies like China and Japan, may exert influential inflationary pressures affecting US monetary policies. [Source](https://www.jpmorgan.com/insights/global-research/outlook/market-outlook)

### Implications for the Market and Content Strategy:

- **Inflation Monitoring**: Content must focus on inflation dynamics, particularly their effect on varied sectors and consumer behavior. Consistent updates on inflation metrics and expert opinions will keep audiences informed about potential market shifts.

- **Investment Strategies**: Educate audiences on diversification strategies and how to adjust asset allocations amidst inflation. Highlighting both traditional and alternative investment vehicles can offer protection against inflationary pressures.
  
- **Political Developments**: Coverage should include analysis of political impacts on market trends. Understanding government policy shifts and their potential market effects will support investors in making well-informed decisions.

- **Global Economic Influences**: Present content on international market trends and their reverberating effects on the US economy. Tie global events to domestic conditions, focusing on regions highlighted in institutional reports like those from J.P. Morgan.

This comprehensive insight aims to guide content strategy, aligning it with future market conditions, to help investors navigate through the challenges of 2024 with a deeper understanding of inflation’s impact on stock markets.

**Stay informed on the latest insights into the 2024 economic landscape by subscribing to our financial newsletter.**

Platform: Twitter
With inflation set to impact the US economy significantly in
2024, aligning your investment strategy is more crucial than
ever. Learn to navigate the complexities of inflation with
our latest insights: [Link to Blog]
------------------------------------------------------------
Platform: LinkedIn
Understanding inflation's trajectory in 2024 is essential
for comprehensively managing investment portfolios. Explore
how political and global trends intertwine with inflationary
dynamics in our latest report. [Link to Blog]
------------------------------------------------------------
Platform: Instagram
Want to stay ahead of inflation trends? Our latest article
unpacks its implications on investment strategies and global
markets. Tap the link in bio for more. #Inflation
#InvestmentStrategy
------------------------------------------------------------
Platform: Facebook
Inflation impacts all corners of the economy, from consumer
spending to stock market dynamics. In 2024, navig